# dh_msra 说明
0. **下载地址：** [Github](https://github.com/SophonPlus/ChineseNlpCorpus/raw/master/datasets/dh_msra/dh_msra.zip)
1. **数据概览：** 5 万多条中文命名实体识别标注数据（[IOB2](https://dl.acm.org/citation.cfm?id=977059) 格式，符合 [CoNLL 2002](https://www.clips.uantwerpen.be/conll2002/ner/) 和 [CRF++](https://taku910.github.io/crfpp/#format) 标准）
2. **推荐实验：** 中文命名实体识别
2. **数据来源：** 不详
3. **原数据集：** [zh-NER-TF](https://github.com/Determined22/zh-NER-TF)，网上搜集，具体作者、来源不详，可能是来自于 MSRA 的语料
4. **加工处理：**
    1. 将原来 2 个文件 (train 和 test) 整合到 1 个文件中

In [1]:
import codecs
import random

import numpy as np

In [2]:
path = './data/'

In [21]:
random.random()

0.0257384541316108

# 1. dh_msra.txt

## 加载数据

In [6]:
def load_iob2(file_path):
    '''加载 IOB2 格式的数据'''
    token_seqs = []
    label_seqs = []
    tokens = []
    labels = []
    with codecs.open(file_path, 'r', encoding='utf-8') as f:
        for index, line in enumerate(f):
            items = line.strip().split()
            if len(items) == 2:
                token, label = items
                tokens.append(token)
                labels.append(label)
            elif len(items) == 0:
                if tokens:
                    token_seqs.append(tokens)
                    label_seqs.append(labels)
                    tokens = []
                    labels = []
            else:
                print('格式错误。行号：{} 内容：{}'.format(index, line))
                continue
                
    if tokens: # 如果文件末尾没有空行，手动将最后一条数据加入序列的列表中
        token_seqs.append(tokens)
        label_seqs.append(labels)    
        
    return np.array(token_seqs), np.array(label_seqs)


In [7]:
def show_iob2(token_seqs, label_seqs, num=5, shuffle=True):
    '''显示 IOB2 格式数据'''
    if shuffle:
        length = len(token_seqs)
        indexes = [random.randrange(0, length) for i in range(num)] 
        zip_seqs = zip(token_seqs[indexes], label_seqs[indexes])
    else:
        zip_seqs = zip(token_seqs[0:num], label_seqs[0:num])
        
    for tokens, labels in zip_seqs:
        for token, label in zip(tokens, labels):
            print('{}/{} '.format(token, label), end='')
        print('\n')

In [8]:
token_seqs, label_seqs = load_iob2(path+'dh_msra.txt')

In [11]:
lengths = [len(item) for item in token_seqs]

In [12]:
lengths = np.array(lengths)

In [14]:
lengths.mean(), lengths.max()

(42.36792128633182, 100)

In [15]:
np.median(lengths)

38.0

In [4]:
print(len(token_seqs), len(label_seqs))
print()    
show_iob2(token_seqs, label_seqs)

55289 55289

目/O 前/O “/O 继/B-PER 生/I-PER ”/O 共/O 产/O 仔/O 5/O 胎/O ，/O 产/O 下/O 小/O 老/O 虎/O 1/O 8/O 只/O ，/O 堪/O 称/O 虎/O 妈/O 妈/O 中/O 的/O 英/O 雄/O 。/O 

历/O 史/O 的/O 内/O 涵/O 是/O 很/O 丰/O 富/O 的/O ，/O 经/O 典/O 作/O 家/O 的/O 论/O 断/O 固/O 然/O 有/O 其/O 权/O 威/O 性/O 和/O 合/O 理/O 性/O ，/O 但/O 历/O 史/O 学/O 家/O 显/O 然/O 不/O 能/O 局/O 限/O 于/O 此/O 。/O 

5/O 月/O 3/O 0/O 日/O 在/O 中/B-LOC 国/I-LOC 革/I-LOC 命/I-LOC 军/I-LOC 事/I-LOC 博/I-LOC 物/I-LOC 馆/I-LOC 开/O 幕/O 的/O 全/O 国/O 禁/O 毒/O 展/O 览/O ，/O 在/O 社/O 会/O 上/O 引/O 起/O 了/O 强/O 烈/O 的/O 反/O 响/O 。/O 

另/O 外/O ，/O 还/O 有/O 一/O 个/O 惊/O 人/O 的/O 发/O 现/O ：/O 有/O 的/O 发/O 展/O 中/O 国/O 家/O 人/O 均/O 国/O 民/O 资/O 源/O 非/O 常/O 丰/O 富/O ，/O 但/O 发/O 展/O 不/O 起/O 来/O 的/O 原/O 因/O 在/O 于/O 教/O 育/O 水/O 平/O 太/O 低/O 、/O 对/O 技/O 术/O 的/O 理/O 解/O 和/O 把/O 握/O 太/O 低/O 、/O 管/O 理/O 水/O 平/O 太/O 低/O 等/O 等/O ，/O 一/O 句/O 话/O ，/O 智/O 力/O 资/O 本/O 太/O 贫/O 乏/O 。/O 

这/O 还/O 要/O 看/O 进/O 一/O 步/O 深/O 入/O 调/O 查/O 的/O 结/O 果/O 。/O 



分割数据集

In [41]:
def train_test_split(file_path, max_len=64, rate=0.9):
    text = []
    subword_len_counter = 0  # 用于标记句子的长度
    with codecs.open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            items = line.split()
            # 如果有两列，添加到数据集中
            if len(items) == 2:
                
                if subword_len_counter > max_len:
                    text.append("")
                    subword_len_counter = 0
                subword_len_counter += 1
                text.append(" ".join(items))
            # 如果为空行，初始化句子长度为 0
            elif len(items) == 0:
                text.append("")
                subword_len_counter = 0
                continue
            else:
                print('格式错误。行号：{} 内容：{}'.format(index, line))
                continue
    # 分割数据集 共 2406374 行
    train_text = []
    test_text = []
    train_text_flag = True
    for item in text:
        # 判断将下一句化添加到训练集还是验证集
        if item == "":
            random_rate = random.random()
            train_text_flag = True if random_rate <= rate else False
            
        if train_text_flag:
            train_text.append(item)
        else:
            test_text.append(item)
            
    return train_text, test_text


In [42]:
train_text, test_text = train_test_split(path+'dh_msra.txt')

In [43]:
len(train_text), len(test_text)

(2164965, 241409)

In [44]:
def save_txt(fp, text):
    with open(fp, 'w', encoding='utf-8') as f:
        f.write('\n'.join(text))


In [46]:
save_txt('./data/train.txt', train_text)
save_txt('./data/test.txt', test_text)

## 标签说明

| 标签 | 说明 |
| ---- | ---- |
| LOC | 地点 (LOCATION) |
| ORG | 机构 (ORGANIZATION) |
| PER | 人物 (PERSON) |

In [33]:
unique_labels = set([label for labels in label_seqs for label in labels])
print(unique_labels)

{'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O'}

In [34]:
with open('./data/labels.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(unique_labels))